## Classifying Text with Vectors

* Last episode we converted text to vectors and used those vectors to find similar documents. 
* This episode we are going to use those vectors for classification.
* We are also going to use machine learning to make the vectors align with our classification goal.

In [1]:
from typing import List, Tuple
import os.path
from pathlib import Path
import random

import torch
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer, BertConfig

# Install TensorBoard

We are going to use it to graph our results.

`pip install tensorboard`

In [3]:
from torch.utils.tensorboard import SummaryWriter

# Let's get the data

* We are going to take my tweets and classify whether they are about movies or not.
* Since most tweets aren't about movies, we have to down sample. Imbalanced classes are a whole topic that we can cover another time.

In [4]:
home = str(Path.home())
data_dir = os.path.join(home, 'temp')
print("Data directory: ", data_dir)

all_tweets: List[str] = []
with open('jmugan_tweets.txt', 'r') as f:
    for tweet in f:
        tweet = tweet.strip()  # remove newline
        all_tweets.append(tweet)

def is_movie_tweet(tweet: str) -> bool:
    return 'movie' in tweet.lower()

def down_sample(all_tweets: List[str]) -> List[str]:
    down_sampled = []
    num_pos = 0
    for tweet in all_tweets:
        if is_movie_tweet(tweet):
            num_pos += 1
            down_sampled.append(tweet)
        elif random.random() > .95:  # let in 80% of others
            down_sampled.append(tweet)
    print(f"Downsampled dataset: {num_pos} positive tweets out of {len(down_sampled)}")
    return down_sampled

# Normally, we would break the data into train, valid, and test, 
# but this is just a quick video
training_tweets = down_sample(all_tweets)


Data directory:  /Users/jmugan/temp
Downsampled dataset: 45 positive tweets out of 101


# We need to `Dataset` to represent our data set

In [5]:
from torch.utils.data import Dataset

# https://pytorch.org/docs/stable/data.html
# You don't have to load the whole data into memory
class TweetDataset(torch.utils.data.Dataset):

    def __init__(self, tweets: List[str]):
        self.tweets = tweets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, index: int):
        return self.tweets[index]

    #def __iter__(self):
    #    # you could be pulling these from a file instead so the whole thing
    #    # doesn't have to sit in memory, see torch.utils.data.IterableDataset
    #    for tweet in self.tweets:
    #        yield tweet


# We need a `DataLoader` to batch and preprocess our data and pass it to the learner

In [6]:
# recall from last time
def get_tokens(text: str,
               tokenizer: BertTokenizer,
               config: BertConfig) -> List[str]:
    tokens = tokenizer.tokenize(text)
    # make sure it isn't too long
    max_length = config.max_position_embeddings
    tokens = tokens[:max_length-1] # Will add special begin token
    # cls token to hold vector https://huggingface.co/transformers/main_classes/tokenizer.html
    tokens = [tokenizer.cls_token] + tokens
    return tokens



def get_labels(batch: List[str]) -> List[int]:
    """
    Normally you have the labeled data from somewhere, maybe somebody hand labeled it.
    Here, we will just use a simple-stupid function
    """
    labels: List[int] = []
    for tweet in batch:
        if is_movie_tweet(tweet):
            labels.append(1)
        else:
            labels.append(0)
    return labels


# We need a function that puts masks on because the tweets have different lengths
def preprocess_batch(batch: List[str]
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, List[str]]:
    """
    We get a list of batch-size in and we have to convert it to ids and make a mask
    """

    # First tokenize
    tokenized_tweets: List[List[str]] = [get_tokens(tweet, tokenizer, config) 
                                         for tweet in batch]

    # find the max length
    lengths = [len(tokenized_tweet) for tokenized_tweet in tokenized_tweets]
    max_length = max(lengths)

    # get batch size
    batch_size = len(batch)

    # let's make it tensors
    input_data = torch.zeros(batch_size, max_length, dtype=torch.long)
    mask_data = torch.zeros(batch_size, max_length, dtype=torch.long)

    for i, tokenized_tweet in enumerate(tokenized_tweets):
        token_ids = tokenizer.convert_tokens_to_ids(tokenized_tweet)
        tensor = torch.tensor(token_ids)
        input_data[i,:len(token_ids)] = tensor
        mask_data[i,:len(token_ids)] = 1

    # get labels
    labels = get_labels(batch)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return input_data, mask_data, labels_tensor, batch


from torch.utils.data import DataLoader

BATCH_SIZE = 20
train_data_loader = DataLoader(
    TweetDataset(training_tweets),
    batch_size = BATCH_SIZE,
    shuffle = True,
    collate_fn=preprocess_batch
)

# Now let's define our classifier as a PyTorch module
* It takes a Bert model as the encoder and declares a matrix of size `hidden_size x 2` to map the Bert vector down to two dimensions (tweet about movies or not)

In [7]:
class Classifier(torch.nn.Module):
    def __init__(self, encoder: BertModel):
        """
        The init function of a module specifies the model parameters
        """
        super().__init__()
        self.encoder = encoder
        # We have two classes, about movies and not about movies
        self.classes = torch.nn.Linear(encoder.config.hidden_size, 2)

    def forward(self, input_data: torch.Tensor,
                mask_data: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward is called when you call the model instance
        """

        # The mask is so it doesn't pay attention to any tokens that are just 
        # filling up space. You need this because the input tensor is of shape 
        # batch_size x max_len where max_len is the longest tweet in that batch.
        _last_hidden_state, pooler_output = self.encoder(input_data, 
                                                         attention_mask = mask_data)

        # shape batch_size x encoder.config.hidden_size
        vectors = pooler_output

        # shape batch_size x 2
        logits = self.classes(vectors)

        # shape batch_size x 2; do softmax and log to pass 
        # into negative log likelihood later
        log_softmax = F.log_softmax(logits, dim=1)

        # We will use the vectors in the next video
        return log_softmax, vectors


# Let's pull up our Bert model like we did before

In [8]:
model_name = 'bert-base-uncased'
# Need to use the same tokenizer that was used to train the model so that it breaks up words
# into tokens the same way.
tokenizer = BertTokenizer.from_pretrained(model_name)

# This model is huge!!!!!!!!
model = BertModel.from_pretrained(model_name)

# What do I use this for?
config = BertConfig.from_pretrained(model_name)

# We can now instantiate our classifier

In [10]:
classifier = Classifier(model)
classifier.train()

Classifier(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

# Set up our optimizers
Slower learning rate for Bert so we don't change it so much

In [11]:
bert_optimizer = torch.optim.Adam(classifier.encoder.parameters(), lr=0.000005)
head_optimizer = torch.optim.Adam(classifier.classes.parameters(), lr=0.001)

# Time to train the model

In [13]:
# For writing to TensorBoard
summary = SummaryWriter(data_dir)

global_step = 0

for epoch in range(10):

    print("STARTING EPOCH: ",epoch)

    for input_tuple in train_data_loader:

        global_step += 1
        input_data, mask_data, labels, tweets = input_tuple
        log_softmax, _vectors = classifier(input_data, mask_data)
        loss = F.nll_loss(log_softmax, labels)

        # Gradients accumulate and you have to clear them out ever time
        bert_optimizer.zero_grad()
        head_optimizer.zero_grad()

        # Get the gradients with respect to the parameters
        loss.backward()

        # Update the parameters
        bert_optimizer.step()
        head_optimizer.step()

        print(f"Loss: {loss:.2f}")

        preds = log_softmax.argmax(1)

        preds_np = preds.detach().numpy()
        labels_np = labels.detach().numpy()

        batch_size = labels_np.shape[0]

        # noinspection PyTypeChecker
        accuracy = sum(labels_np == preds_np) / batch_size

        print(f"Predictions: {preds_np}, Labels: {labels_np}, Accuracy: {accuracy}")
        print(tweets)

        summary.add_scalar('loss_train', loss, global_step=global_step)
        summary.add_scalar('accuracy', accuracy, global_step=global_step)

STARTING EPOCH:  0
Loss: 0.72
Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], Labels: [1 1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0], Accuracy: 0.6
['Isn\'t it weird that an actor is "in" a movie but "on" a TV show?', 'Love it when movies show you what it would be like to be someone else. Saw Last Train Home. Great movie about migrant workers in China.', 'Amazing how black socks change to blue when you take them out of the dresser drawer.', 'My kids saw a typewriter today and were blown away. "What IS that thing?" Like it was from an alien world.', "I've largely cut sugar from my diet this week. Remember that Paul Giamatti movie Cold Souls? Me too.", 'Recently watched the movie The Bay. Anyone want to go for a swim?', 'Coat hangers are very poorly behaved objects.', 'My inbox now has 0 messages. It feels good to get organized.', "I wish my financial/retirement companies would stop sending me mail. With all these unnecessary words, I'm likely to miss important ones.", 'Remember that 

Loss: 0.73
Predictions: [1 1 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1], Labels: [0 1 0 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1], Accuracy: 0.4
["I successfully resurrected my wife's old laptop by installing Ubuntu. My work here is done.", "Finally saw Sleep Dealer (Traficante de Suenos). If you like dystopian movies about migrant works and drones, this one's for you. I loved it", 'Finally saw Napoleon Dynamite. Love how you can always hear the birds chirping in the background. And the steaks.', 'Talking about the 6 Star Wars movies with my kids. Terms like "first one" and "last one" result in limitless confusion.', "My son (7) wants to know why he can't have a credit card.", "I bet that primitive people used to just sit around and tell stories about monsters. Anyway, I'm off to the movies.", 'It would be nice if we could abandon all of this inane talk about celebrities and go back to gossiping about people that we actually know.', "Hate wearing a belt, but I need to dress fancy for work. I'm wait

Loss: 0.78
Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], Labels: [0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1], Accuracy: 0.4
["I hate when I have to get up early. That's enough, but my body will often get me up even earlier than that for no good reason.", 'Sprinkling unexpected words into your prose can keep it fresh but can also distract the reader. #Trade-offs #Writing', "Last one in the office. I'm riding a scooter down an empty hall, and I'm suddenly reminded of the Big Wheel scene from The Shining.", "One bummer about getting older is that I've already seen every new movie that comes out.", "According to toddlers, a cat's tail was made for pulling. The perfect Gibsonian affordance.", "My son (7) wants to know why he can't have a credit card.", 'Amazing how black socks change to blue when you take them out of the dresser drawer.', "Surprisingly, the search feature on Google docs doesn't work very well. If only they had access to search experts.", 'I\'m used to "twenty years 

Loss: 0.68
Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0], Labels: [0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0], Accuracy: 0.6
["I hate when I write a note to myself but then later don't remember what the note refers to.", "After having watched many Stars Wars movies with my kids lately, I've decided that R2-D2 is the unsung hero. The AI is strong in him.", "I like a lot of kids movies, but I just can't get into these Ice Age films.", "Companies should realize that surveys must be short. I'm not going to click on  25 ovals, I just want to say the hotel carpet was dirty.", "I've largely cut sugar from my diet this week. Remember that Paul Giamatti movie Cold Souls? Me too.", 'I\'m used to "twenty years ago" meaning the 1970s, but that was twenty years ago.', 'Told my wife I rented a movie about an automobile tire that comes to life and murders people. She just walked out of the room. #Marriage', "Was just explaining @klout and how a person's influence is measured relative to the ma

Loss: 0.76
Predictions: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1], Labels: [0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0], Accuracy: 0.55
['I love the smell of coffee in the morning. It smells like victory.', 'My inbox now has 0 messages. It feels good to get organized.', "The movie I watched about a tire that comes to life and starts killing people wasn't as good as I had hoped it would be.", 'I love how a good book becomes part of your mental life forever.', 'Isn\'t it weird that an actor is "in" a movie but "on" a TV show?', "My stomach is uncomfortably full, but I am still distracted by hunger. What's that story about the guy pushing the rock up the hill?", 'The weird thing about the Lord of the Rings movies is that they all seem to just end. Like they ran out of film or something. #TheHobbit', "There must be a special place in hell for the people who decided you can't skip movie previews on DVD.", "I hate when I have to get up early. That's enough, but my body will often get me up even

...

Loss: 0.42
Predictions: [0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0], Labels: [0 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0], Accuracy: 0.9
['Science moves forward by more accurate predictions. Math by new theorems. How does philosophy move forward?', 'I love getting reminders from Microsoft Outlook telling me that I am 23 hours late for a meeting.', 'Went to Costco for the first time not too long ago. That place is a disaster movie.', 'Sprinkling unexpected words into your prose can keep it fresh but can also distract the reader. #Trade-offs #Writing', "Camping in the living room eating S'mores and watching a movie. It's surprising that they don't have more safety railings on the Death Star.", "There must be a special place in hell for the people who decided you can't skip movie previews on DVD.", 'I love it when movies about ghosts are "based on a true story."', "I used to go one way to work. But lately, I've been asking Google Maps, and it sends me another way. I'm starting to see its wisdo

Loss: 0.43
Predictions: [1 1 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1], Labels: [1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1], Accuracy: 0.85
['I wonder why we raise our kids on movies with clear good guys and bad guys, when for the most part, the real world is just people', 'Just saw Black Swan. Man, that was stressful. I need to relax, maybe watch a movie or something.', "For picking movies, I've learned that not only must I enjoy it for 2 hours, but I must also enjoy having it in my head for days after.", 'I just wrote the word "print" and Google Docs flagged it as misspelled. It wanted me to write "printf". How cool is that?', 'One of my superpowers is the ability to abandon a book or movie halfway through.', 'Should the streets be widened to help emergency vehicles? Of course. Should my yard be shortened to widen the street? Get off my property.', 'Just saw Contagion. I thought it was pretty good. I like that it stayed with the big picture. Not as sappy as most disaster movies.', 'Coat han

Loss: 0.17
Predictions: [1], Labels: [1], Accuracy: 1.0
["Some movies don't seem to have enough material to fill up the trailer. Don't know what they do for the other 1 hour and 28 minutes."]
STARTING EPOCH:  8
Loss: 0.28
Predictions: [1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1], Labels: [1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1], Accuracy: 0.9
["There must be a special place in hell for the people who decided you can't skip movie previews on DVD.", "My stomach is uncomfortably full, but I am still distracted by hunger. What's that story about the guy pushing the rock up the hill?", 'Wrestling with my boys. Good to have a little cave-man time before I go back to work tomorrow.', 'I love reading the reviews for kids movies. "... the archetypes do not resonate as one would expect given the ..." Kids want burp jokes.', "I hate when I have to get up early. That's enough, but my body will often get me up even earlier than that for no good reason.", "It's amazing how much joy in life comes from 

Loss: 0.28
Predictions: [0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1], Labels: [0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1], Accuracy: 0.9
['I\'ve been in the workforce for 15 years, and I still can\'t get in the habit of saying "good morning" instead of "hey" or "what\'s up?"', 'I wonder why we raise our kids on movies with clear good guys and bad guys, when for the most part, the real world is just people', 'Just saw Limitless. Great movie. But he said doubling your money every day was too slow. Clearly, even with NZT, he was still limited.', 'One of my superpowers is the ability to abandon a book or movie halfway through.', 'Compared to the cold and hungry existence lived by our ancestors, our society is almost perfect. I tell myself that when I buy printer ink.', 'Recently watched the movie The Bay. Anyone want to go for a swim?', 'Young people sometimes read the same books and watch the same movies multiple times. When we get older we stop doing that. Why?', "Last one in the office. I'm 

Loss: 0.20
Predictions: [0 0 1 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0], Labels: [0 1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0], Accuracy: 0.9
["My son (7) wants to know why he can't have a credit card.", "I've never been one to see symbolism in books or movies. I like to take fictional stories at face value, as if they were my own experience.", 'Talking about the 6 Star Wars movies with my kids. Terms like "first one" and "last one" result in limitless confusion.', 'When they show scientists in movies, they never show them making PowerPoint slides.', 'I love how a good book becomes part of your mental life forever.', "With a big bucket of popcorn, it doesn't matter what the movie is. You're happy.", 'HT @TheOnion: 5-Year-Old Critics Agree: Movie "Cars" Only Gets Better After 40th Viewing', "Last one in the office. I'm riding a scooter down an empty hall, and I'm suddenly reminded of the Big Wheel scene from The Shining.", 'Went to Costco for the first time not too long ago. That place is a disas

# Save the model and print out the parameters

In [14]:
# Let's save the model (we'll use it in the next video)
torch.save(classifier.state_dict(), os.path.join(data_dir,'classifier_model.pt'))

# Let's print out the state dict
print(classifier.state_dict().keys())

odict_keys(['encoder.embeddings.word_embeddings.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.LayerNorm.bias', 'encoder.encoder.layer.0.attention.self.query.weight', 'encoder.encoder.layer.0.attention.self.query.bias', 'encoder.encoder.layer.0.attention.self.key.weight', 'encoder.encoder.layer.0.attention.self.key.bias', 'encoder.encoder.layer.0.attention.self.value.weight', 'encoder.encoder.layer.0.attention.self.value.bias', 'encoder.encoder.layer.0.attention.output.dense.weight', 'encoder.encoder.layer.0.attention.output.dense.bias', 'encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.encoder.layer.0.intermediate.dense.weight', 'encoder.encoder.layer.0.intermediate.dense.bias', 'encoder.encoder.layer.0.output.dense.weight', 'encoder.encoder.layer.0.output.dense.bias', 'encoder.encoder.la

# Look at the curves in TensorBoard

`tensorboard --logdir=./`

then go to http://localhost:6006/

You should see graphs that look like this
![Tensorboard graphs](./tensorboard.png)

# In the next video, we'll look at these vectors in 3D space in TensorBoard and compare them with the vectors created in episode 1

In [17]:
import numpy as np
import pickle

classifier.eval()  # needed when the model uses things like dropout


all_vectors_np = []
all_tweets = []
for input_tuple in train_data_loader:

        input_data, mask_data, labels, tweets = input_tuple
        _log_softmax, vectors = classifier(input_data, mask_data)
        
        vectors_np = vectors.detach().numpy()
        all_vectors_np.append(vectors_np)
        all_tweets.extend(tweets)

all_vecs = np.concatenate(all_vectors_np, axis=0)
print(all_vecs.shape)


custom_data = {'tweets':all_tweets, 'vecs':all_vecs}

custom_vec_pickle_file = os.path.join(data_dir, 'custom_vecs.pkl')
with open(custom_vec_pickle_file, 'wb') as f:
        pickle.dump(custom_data, f)



(101, 768)
